In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
# Librerias
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# Carga de datos
from google.colab import drive
drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore.csv"
reviews_file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore_user_reviews.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
google_play_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(file_path, header=True, inferSchema=True)
user_reviews_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(reviews_file_path, header=True, inferSchema=True)
google_play_data = google_play_data.rdd
user_reviews_data = user_reviews_data.rdd
# Filtra las filas en las que la categoría no sea "1.9" y crea un nuevo RDD sin esa fila
google_play_data = google_play_data.filter(lambda x: x['Category'] != "1.9")


# 6 - Queremos saber cuánto pesaría si quisiéramos bajar todas las apps de un género, para todos los géneros.Para eso se pide: Calcular separado por géneros, cuanto pesarian todas las apps que tienen ese género (Tener en cuenta que si una app tiene acción y arte, su peso cuenta para ambos géneros) (⭐⭐)

Filtrar las aplicaciones que tienen "Varies with device" en la columna "Size"

In [ ]:
filtered_data = google_play_data.filter(lambda x: x['Size'] != 'Varies with device' and str(x['Size'] != 'nan') )

Filtrar el RDD para obtener solo las columnas 'Genres' y 'Size'

In [ ]:
genres_and_sizes = filtered_data.map(lambda x: (x['Genres'], x['Size']))

Separar las aplicaciones por sus generos

In [ ]:
genre_sizes = genres_and_sizes.flatMap(lambda x: [(genre.strip(), x[1]) for genre in x[0].split(';')])

Función para convertir tamaños en KB y MB a MB

In [ ]:
def convert_size_to_mb(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024  # Convertir KB a MB
    else:
        return None  # En caso de valores desconocidos

Convertir el tamaño de las aplicaciones a un formato numérico en MB

In [ ]:
genre_sizes = genre_sizes.map(lambda x: (x[0], convert_size_to_mb(x[1])))

Calcular el tamaño total de las aplicaciones para cada género

In [ ]:
total_sizes_by_genre = genre_sizes.reduceByKey(lambda x, y: x + y )

El tamaño se va a mostrar en Mb

In [ ]:
total_sizes_by_genre.collect()

[('Art & Design', 812.0000000000002),
 ('Pretend Play', 4082.2000000000003),
 ('Creativity', 1236.9),
 ('Action & Adventure', 5757.099999999999),
 ('Auto & Vehicles', 1502.760546875),
 ('Beauty', 648.4000000000001),
 ('Books & Reference', 2740.10703125),
 ('Business', 5788.750976562499),
 ('Comics', 687.7283203125002),
 ('Communication', 3117.9033203124995),
 ('Dating', 3524.6175781250017),
 ('Education', 14758.330273437494),
 ('Brain Games', 1967.6999999999998),
 ('Entertainment', 9057.689843750006),
 ('Music & Video', 865.3000000000001),
 ('Events', 795.9261718749999),
 ('Finance', 5503.991015624996),
 ('Food & Drink', 2218.5),
 ('Health & Fitness', 6148.976562500001),
 ('House & Home', 1086.5406249999999),
 ('Libraries & Demo', 908.50693359375),
 ('Lifestyle', 5090.168359375),
 ('Adventure', 3411.4),
 ('Arcade', 8858.800000000001),
 ('Casual', 8482.000000000004),
 ('Card', 1348.51328125),
 ('Strategy', 5258.900000000001),
 ('Action', 17347.3279296875),
 ('Puzzle', 5307.604296875001)